In [1]:
import os
from warnings import simplefilter

simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)

import pandas as pd
from natsort import natsort

from utilities import plot_figure_heatmap

Constants.

In [2]:
ROOT = '.'
DATA_FOLDER = f'{ROOT}/data'
OUT_FOLDER = f'{ROOT}/results'
os.makedirs(OUT_FOLDER, exist_ok=True)
FILE = f"{DATA_FOLDER}/Supp. File 1.xlsx"

Read the data.

In [3]:
data = pd.read_excel(FILE, sheet_name=0)
labels = pd.read_excel(FILE, sheet_name=1, usecols=['InChIKey', 'binaryDILI'])
data = data.merge(labels, on='InChIKey')
data = data[data.Inclusion == 'Included']
data = data.dropna()

Organize the data.

In [4]:
data = pd.concat([data.iloc[:, :5], data[['binaryDILI']], data.iloc[:, 5:-1]], axis=1)

# Select Pos1i for all but for ICAM select Diff1i
data = data.loc[:, (data.columns.str.endswith('GfpPos1i') & ~data.columns.str.startswith('icam1')) | (data.columns.str.endswith('GfpDiff1i') & data.columns.str.startswith('icam1')) | data.columns.isin(data.columns[:6])]
data['binaryDILI'] = data['binaryDILI'].replace({0: 'Negative', 1: 'Positive'})
data.index = data.Name.str.lower()

data.columns = data.columns[:6].tolist() + pd.Series(data.columns[6:]).str.split('_').apply(
    lambda x: f"{x[0]}_{x[2]}_{x[1]}_{'_'.join(x[3:])}").tolist()
data = data[data.columns[:6].tolist() + natsort.natsorted(data.columns[6:])]
data['binaryDILI'] = pd.Categorical(data.binaryDILI, categories=["Negative", "Positive"], ordered=True)

melt = pd.DataFrame(pd.Series(data.columns[6:]).str.split('_').apply(lambda x: x[:3]).values.tolist(),
                    columns=['Cell line', 'Time point', 'Concentration'])
melt = melt.replace({'tp24': '24', 'tp48': '48', 'tp72': '72'})
melt = melt.replace(
    {'cmax1': '1 CMAX', 'cmax5': '5 CMAX', 'cmax10': '10 CMAX', 'cmax25': '25 CMAX', 'cmax50': '50 CMAX',
     'cmax100': '100 CMAX'})
melt['Cell line'] = melt['Cell line'].str.upper()
melt['Cell line'] = pd.Categorical(melt['Cell line'], ordered=True,
                                   categories=['SRXN1', 'HMOX1', 'CHOP', 'BIP', 'P21', 'BTG2', 'HSPA1B', 'ICAM1'])

melt.index = data.columns[6:]

Obtain Figure 1.


In [5]:
plot_figure_heatmap(data.copy(), melt.copy(), folder=OUT_FOLDER, colormap='RdYlBu_r')

In [6]:
# Comment out the next lines to obtain other heatmaps.
# readouts = pd.Series(data.columns[6:-1].str.split('_')).apply(lambda x: '_'.join(x[3:])).unique()
# pbar = tqdm(natsort.natsorted(readouts))
# for readout in pbar:
#     pbar.set_description(readout)
#     plot_heatmap(data.copy(), readout, OUT_FOLDER)